In [1]:
import os
import shutil

import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join
import json 

import plotly
import plotly.express as px
import plotly.io as pio

# featureCounts

In [2]:
experiment_id = "NS-23.0061"

IDR = "nIDR" # "standard"  

metadata_subdir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/analysis/metadata/".format(experiment_id))
print('metadata_subdir exists:{}\n'.format(os.path.exists(metadata_subdir)))

PEAKS_dir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/alignment/macs2/".format(experiment_id))
print('PEAKS_dir exists:{}\n'.format(os.path.exists(PEAKS_dir)))

BAM_DIR = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/alignment/bam/".format(experiment_id))
print('BAM_DIR exists:{}\n'.format(os.path.exists(BAM_DIR)))

idr_bed_dir = "../idr_bed/"
print('idr_bed_dir exists:{}\n'.format(os.path.exists(idr_bed_dir)))

metadata_subdir exists:True

PEAKS_dir exists:True

BAM_DIR exists:True

idr_bed_dir exists:True



In [3]:
def create_dir(name_dir):
    if os.path.exists(name_dir):
        shutil.rmtree(name_dir)
    os.mkdir(name_dir)
    return name_dir

In [4]:
merged_IDR_narrowPeak_dir = create_dir("../merged_IDR_narrowPeak/")
print(merged_IDR_narrowPeak_dir)

featureCounts_dir = create_dir("../featureCounts/")
print(featureCounts_dir)

../merged_IDR_narrowPeak/
../featureCounts/


In [5]:
if IDR == "standard":
    IDR_narrowPEAK_dir = create_dir("../IDR_narrowPEAK/")
    print(IDR_narrowPEAK_dir)
    
    macs2_dir = "~/fht.samba.data/experiments/ATACseq/{}/alignment/macs2/*IDR0.1.narrowPeak.gz".format(experiment_id) 
    
    !cp {macs2_dir} {IDR_narrowPEAK_dir}
    !!gunzip {IDR_narrowPEAK_dir}/*.gz*

In [8]:
def read_contrast_dict(metadata_subdir, experiment_id): 
    output_filename = experiment_id + '_contrast_dict.json'
    print("output_filename : {}".format(output_filename))

    output_filepath = os.path.join(metadata_subdir, output_filename)
    print("output_filepath : {}".format(output_filepath))
    
    # Opening JSON file
    with open(output_filepath) as json_file:
        contrast_dict = json.load(json_file)
        
    return contrast_dict

contrast_dict = read_contrast_dict(metadata_subdir, experiment_id)
# contrast_dict

output_filename : NS-23.0061_contrast_dict.json
output_filepath : /home/fuzan/fht.samba.data/experiments/ATACseq/NS-23.0061/analysis/metadata/NS-23.0061_contrast_dict.json


In [9]:
# contrast_dict = dict(filter(lambda item:('KMS20' in item[0]) or ('KO52' in item[0]), contrast_dict.items()))
# contrast_dict

In [10]:
def merge_narrowPeak(contrast_dict, IDR):
    if IDR == 'standard':
        print('Standard IDR version')
    if IDR == 'nIDR':
        print('nIDR version')        

    contrast_list = list(set(contrast_dict.keys()))
    print("\ncontrast_list: \n{}\n".format(contrast_list))

    for contrast in list(contrast_dict.keys()):
            print("contrast: {}".format(contrast))

            # # 1) combine test+grp and negative_cntrl_grp
            # if IDR == 'standard':
            #     test_group_peak_file = contrasts_df.loc[list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].index)[0], 'test_peak_file']
            #     negative_ctrl_grp_peak_file = contrasts_df.loc[list(contrasts_df[contrasts_df['file_friendly_name'] == contrast].index)[0], 'negative_ctrl_peak_file']
            #     print(test_group_peak_file)
            #     print(negative_ctrl_grp_peak_file)
            #     !grep -h '^chr'  {IDR_narrowPEAK_dir}{test_group_peak_file} {IDR_narrowPEAK_dir}{negative_ctrl_grp_peak_file} > {merged_IDR_narrowPeak_dir}{contrast}.combined.narrowPeak

            if IDR == 'nIDR': 
                test_group = list(contrast_dict[contrast]['test_group'].keys())[0]
                sample_test_group = list(contrast_dict[contrast]['test_group'].values())[0]
                print('test_group: {} -> {}'.format(test_group, sample_test_group))

                negative_ctrl_grp = list(contrast_dict[contrast]['negative_ctrl_grp'].keys())[0]
                sample_negative_ctrl_grp = list(contrast_dict[contrast]['negative_ctrl_grp'].values())[0]
                print("sample_negative_ctrl_grp: {} -> {}\n".format(negative_ctrl_grp, sample_negative_ctrl_grp))
                
                cmd_cat = """grep -h '^chr'  {idr_bed_dir}{test_group}.IDR.narrowPeak {idr_bed_dir}{negative_ctrl_grp}.IDR.narrowPeak > {merged_IDR_narrowPeak_dir}{contrast}.combined.narrowPeak""".format(idr_bed_dir=idr_bed_dir, test_group=test_group, negative_ctrl_grp=negative_ctrl_grp, contrast=contrast, merged_IDR_narrowPeak_dir=merged_IDR_narrowPeak_dir)
                # print(cmd_cat)
                ! {cmd_cat}

            # 2) sort by chromosome positions (chr, start)
            cmd_sort = """sort -k1,1V -k2,2n -k3,3n {merged_IDR_narrowPeak_dir}{contrast}.combined.narrowPeak > {merged_IDR_narrowPeak_dir}{contrast}.sorted.narrowPeak""".format(merged_IDR_narrowPeak_dir=merged_IDR_narrowPeak_dir, contrast=contrast)
            # print(cmd_sort)
            ! {cmd_sort}
            
            # 3) bedtools merge
            cmd_merge = """bedtools merge -d 50 -i  {merged_IDR_narrowPeak_dir}{contrast}.sorted.narrowPeak > {merged_IDR_narrowPeak_dir}{contrast}.merged.narrowPeak""".format(merged_IDR_narrowPeak_dir=merged_IDR_narrowPeak_dir, contrast=contrast)
            # print(cmd_merge)
            !{cmd_merge}
            
            # 6) add header and convert merged.bed file to saf file for each contrast
            cmd_saf = """awk 'BEGIN{{FS=OFS="\t"; print "GeneID\tchr\tstart\tend\tstrand"}} {{print $1":"$2"-"$3, $1, $2+1, $3, "."}}'  {merged_IDR_narrowPeak_dir}{contrast}.merged.narrowPeak > {merged_IDR_narrowPeak_dir}{contrast}.merged.saf""".format(merged_IDR_narrowPeak_dir=merged_IDR_narrowPeak_dir, contrast=contrast)
            # print(cmd_saf)
            ! {cmd_saf}

            
merge_narrowPeak(contrast_dict, IDR)

nIDR version

contrast_list: 
['ARID1AKO_HCT116_WT_HCT116', 'ARID1BKD_HCT116_WT_HCT116', 'ARID1AKO_ARID1BKD_HCT116_WT_HCT116', 'ARID1BKD_TOV21G_WT_TOV21G', 'ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116']

contrast: ARID1BKD_HCT116_WT_HCT116
test_group: ARID1BKD_HCT116 -> ['SRR5876160', 'SRR5876161']
sample_negative_ctrl_grp: WT_HCT116 -> ['SRR5876158', 'SRR5876159']

contrast: ARID1AKO_HCT116_WT_HCT116
test_group: ARID1AKO_HCT116 -> ['SRR5876162', 'SRR5876163']
sample_negative_ctrl_grp: WT_HCT116 -> ['SRR5876158', 'SRR5876159']

contrast: ARID1AKO_ARID1BKD_HCT116_WT_HCT116
test_group: ARID1AKO_ARID1BKD_HCT116 -> ['SRR5876164', 'SRR5876165']
sample_negative_ctrl_grp: WT_HCT116 -> ['SRR5876158', 'SRR5876159']

contrast: ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116
test_group: ARID1AKO_ARID1BKD_HCT116 -> ['SRR5876164', 'SRR5876165']
sample_negative_ctrl_grp: ARID1AKO_HCT116 -> ['SRR5876162', 'SRR5876163']

contrast: ARID1BKD_TOV21G_WT_TOV21G
test_group: ARID1BKD_TOV21G -> ['SRR5876663', 'SRR58766

In [11]:
def compute_featureCounts(contrast_dict):    

    contrast_list = list(set(contrast_dict.keys()))
    print("\ncontrast_list: \n{}\n".format(contrast_list))

    for contrast in list(contrast_dict.keys()):
            print("contrast: {}".format(contrast))
            test_group = list(contrast_dict[contrast]['test_group'].keys())[0]
            sample_test_group = list(contrast_dict[contrast]['test_group'].values())[0]
            print('test_group: {} -> {}'.format(test_group, sample_test_group))

            negative_ctrl_grp = list(contrast_dict[contrast]['negative_ctrl_grp'].keys())[0]
            sample_negative_ctrl_grp = list(contrast_dict[contrast]['negative_ctrl_grp'].values())[0]
            print("sample_negative_ctrl_grp: {} -> {}\n".format(negative_ctrl_grp, sample_negative_ctrl_grp))


            # 7) Call featureCounts on saf file for each contrast      
            bam_list = [BAM_DIR + group + '.small.chrsorted.BAM' for group in sample_test_group + sample_negative_ctrl_grp]
            bam_list_bash = '\t'.join(bam_list)
            cmd_featureCounts = """featureCounts -a {merged_IDR_narrowPeak_dir}{contrast}.merged.saf -F SAF  -o {merged_IDR_narrowPeak_dir}{contrast}.orig.txt {bam_list_bash}""".format(contrast=contrast, merged_IDR_narrowPeak_dir=merged_IDR_narrowPeak_dir, bam_list_bash=bam_list_bash)
            # print(cmd_featureCounts)
            ! {cmd_featureCounts}

            # 8) Remove header 
            !sed -i '1,2d' {merged_IDR_narrowPeak_dir}{contrast}.orig.txt

            # 9) reformat header for DPA logFC computation
            sample_list_bash = '\t'.join(sample_test_group + sample_negative_ctrl_grp)
            sample_number = [1,2,3,4] + list(np.arange(7, 7+ len(sample_test_group + sample_negative_ctrl_grp), 1))
            sample_number_str = [str(i) for i in sample_number]
            sample_number_bash = '$' + ', $'.join(sample_number_str)
            cmd_header = """awk 'BEGIN{{FS=OFS="\t"; print "peak_id\tchr\tstart\tend\t{sample_list_bash}"}} {{print {sample_number_bash}}}' {merged_IDR_narrowPeak_dir}{contrast}.orig.txt > {featureCounts_dir}{contrast}.txt""".format(sample_list_bash=sample_list_bash, sample_number_bash=sample_number_bash, merged_IDR_narrowPeak_dir=merged_IDR_narrowPeak_dir,featureCounts_dir=featureCounts_dir, contrast=contrast)
            # print(cmd_header)
            ! {cmd_header}
            
compute_featureCounts(contrast_dict)


contrast_list: 
['ARID1AKO_HCT116_WT_HCT116', 'ARID1BKD_HCT116_WT_HCT116', 'ARID1AKO_ARID1BKD_HCT116_WT_HCT116', 'ARID1BKD_TOV21G_WT_TOV21G', 'ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116']

contrast: ARID1BKD_HCT116_WT_HCT116
test_group: ARID1BKD_HCT116 -> ['SRR5876160', 'SRR5876161']
sample_negative_ctrl_grp: WT_HCT116 -> ['SRR5876158', 'SRR5876159']


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 4 BAM f